In [2]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
!pip install numpy matplotlib pandas torch==1.7.1+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy matplotlib pandas torch torchvision torchaudio

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cpu/torch-1.7.1%2Bcpu-cp39-cp39-linux_x86_64.whl (159.4 MB)
ERROR: Could not find a version that satisfies the requirement torchvision==0.8.1+cpu (from versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.8.2, 0.8.2+cpu, 0.8.2+cu101, 0.8.2+cu110, 0.8.2+cu92, 0.9.0, 0.9.0+cpu, 0.9.0+cu101, 0.9.0+cu111, 0.9.1, 0.9.1+cpu, 0.9.1+cu101, 0.9.1+cu102, 0.9.1+cu111, 0.10.0, 0.10.0+cpu, 0.10.0+cu102, 0.10.0+cu111, 0.10.0+rocm4.1, 0.10.0+rocm4.2, 0.10.1, 0.10.1+cpu, 0.10.1+cu102, 0.10.1+cu111, 0.10.1+rocm4.1, 0.10.1+rocm4.2, 0.11.0, 0.11.0+cpu, 0.11.0+cu102, 0.11.0+cu111, 0.11.0+cu113, 0.11.0+rocm4.1, 0.11.0+rocm4.2, 0.11.1, 0.11.1+cpu, 0.11.1+cu102, 0.11.1+cu111, 0.11.1+cu113, 0.11.1+rocm4.1, 0.11.1+rocm4.2, 0.11.2, 0.11.2+cpu, 0.11.2+cu102, 0.11.2+cu111, 0.11.2+cu113, 0.11.2+rocm4.1, 0.11.2+rocm4.2, 0.11.3, 0.11.3+cpu, 0.11.3+cu102, 0.11.3+cu111,

In [3]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [4]:
project_name='02-insurance-linear-regression' # will be used by jovian.commit

## Step 1: Download and explore the data

Let us begin by downloading the data. We'll use the `download_url` function from PyTorch to get the data as a CSV (comma-separated values) file. 

In [5]:
DATASET_URL = "https://gist.github.com/BirajCoder/5f068dfe759c1ea6bdfce9535acdb72d/raw/c84d84e3c80f93be67f6c069cbdc0195ec36acbd/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

Using downloaded and verified file: ./insurance.csv


To load the dataset into memory, we'll use the `read_csv` function from the `pandas` library. The data will be loaded as a Pandas dataframe. See this short tutorial to learn more: https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/

In [6]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


We're going to do a slight customization of the data, so that you every participant receives a slightly different version of the dataset. Fill in your name below as a string (enter at least 5 characters)

In [7]:
your_name = 'alfiya' # at least 5 characters

The `customize_dataset` function will customize the dataset slightly using your name as a source of random numbers.

In [8]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)
    return dataframe

In [9]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,charges
27,55,female,35.3970,2,no,12514.004895
997,63,female,39.7980,0,no,14165.727870
162,54,male,42.7680,1,no,10659.563040
824,60,male,26.2656,0,no,12774.076896
392,48,male,33.9606,1,no,9143.341761


Let us answer some basic questions about the dataset. 


**Q1: How many rows does the dataset have?**

In [10]:
num_rows = dataframe.shape[0]
print(num_rows)

1271


**Q2: How many columns doe the dataset have**

In [11]:
num_cols = dataframe.shape[1]
print(num_cols)

6


**Q3: What are the column titles of the input variables?**

In [12]:
input_cols = list(dataframe.columns[:5])
input_cols

['age', 'sex', 'bmi', 'children', 'smoker']

**Q4: Which of the input columns are non-numeric or categorial variables ?**

Hint: `sex` is one of them. List the columns that are not numbers.

In [13]:
categorical_cols = ['sex','smoker']
categorical_cols

['sex', 'smoker']

**Q5: What are the column titles of output/target variable(s)?**

In [14]:
output_cols = ['charges']

**Q: (Optional) What is the minimum, maximum and average value of the `charges` column? Can you show the distribution of values in a graph?**
Use this data visualization cheatsheet for referece: https://jovian.ai/aakashns/dataviz-cheatsheet

In [15]:
# Write your answer here
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1271 entries, 27 to 1205
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1271 non-null   int64  
 1   sex       1271 non-null   object 
 2   bmi       1271 non-null   float64
 3   children  1271 non-null   int64  
 4   smoker    1271 non-null   object 
 5   charges   1271 non-null   float64
dtypes: float64(2), int64(2), object(2)
memory usage: 69.5+ KB


In [16]:
dataframe.max()
dataframe.charges.max()

65045.83657020001

In [17]:
dataframe.charges.min()

1144.311378

In [18]:
dataframe.charges.mean()

13580.479211857553

Remember to commit your notebook to Jovian after every step, so that you don't lose your work.

In [19]:
!pip install jovian --upgrade -q

In [20]:
import jovian

In [21]:
jovian.commit()


<IPython.core.display.Javascript object>

[jovian] Updating notebook "alfiya-a/02-insurance-linear-regression" on https://jovian.com
[jovian] Committed successfully! https://jovian.com/alfiya-a/02-insurance-linear-regression


'https://jovian.com/alfiya-a/02-insurance-linear-regression'

## Step 2: Prepare the dataset for training

We need to convert the data from the Pandas dataframe into a PyTorch tensors for training. To do this, the first step is to convert it numpy arrays. If you've filled out `input_cols`, `categorial_cols` and `output_cols` correctly, this following function will perform the conversion to numpy arrays.

In [22]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

Read through the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to understand how we're converting categorical variables into numbers.

In [23]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[55.    ,  0.    , 35.397 ,  2.    ,  0.    ],
        [63.    ,  0.    , 39.798 ,  0.    ,  0.    ],
        [54.    ,  1.    , 42.768 ,  1.    ,  0.    ],
        ...,
        [58.    ,  1.    , 34.5708,  1.    ,  0.    ],
        [32.    ,  0.    , 47.7576,  0.    ,  0.    ],
        [35.    ,  1.    , 19.2888,  1.    ,  0.    ]]),
 array([[12514.004895],
        [14165.72787 ],
        [10659.56304 ],
        ...,
        [12185.558418],
        [ 4074.061356],
        [ 5218.830408]]))

**Q6: Convert the numpy arrays `inputs_array` and `targets_array` into PyTorch tensors. Make sure that the data type is `torch.float32`.**

In [24]:
inputs = torch.tensor(inputs_array,dtype=torch.float32)
targets = torch.tensor(targets_array,dtype=torch.float32)

In [25]:
inputs.dtype, targets.dtype

(torch.float32, torch.float32)

Next, we need to create PyTorch datasets & data loaders for training & validation. We'll start by creating a `TensorDataset`.

In [26]:
dataset = TensorDataset(inputs, targets)

**Q7: Pick a number between `0.1` and `0.2` to determine the fraction of data that will be used for creating the validation set. Then use `random_split` to create training & validation datasets.**

In [27]:
val_percent = 0.15 # between 0.1 and 0.2
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size


train_ds, val_ds = random_split(dataset, [train_size, val_size]) # Use the random_split function to split dataset into 2 parts of the desired length

Finally, we can create data loaders for training & validation.

**Q8: Pick a batch size for the data loader.**

In [28]:
batch_size = 8

In [29]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Let's look at a batch of data to verify everything is working fine so far.

In [30]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[64.0000,  0.0000, 24.8292,  0.0000,  1.0000],
        [62.0000,  0.0000, 27.0000,  0.0000,  0.0000],
        [43.0000,  1.0000, 35.2080,  2.0000,  0.0000],
        [31.0000,  0.0000, 25.4880,  2.0000,  0.0000],
        [30.0000,  1.0000, 26.3520,  3.0000,  1.0000],
        [18.0000,  0.0000, 45.6192,  0.0000,  1.0000],
        [27.0000,  0.0000, 25.0668,  1.0000,  0.0000],
        [51.0000,  1.0000, 39.9600,  0.0000,  0.0000]])
targets: tensor([[27578.6719],
        [13720.1445],
        [ 7590.3311],
        [ 5030.2798],
        [18624.4004],
        [39568.5391],
        [ 3633.1267],
        [ 8974.5645]])


Let's save our work by committing to Jovian.

In [31]:
jovian.commit(project=project_name, environment=None)

<IPython.core.display.Javascript object>

[jovian] Updating notebook "alfiya-a/02-insurance-linear-regression" on https://jovian.com
[jovian] Committed successfully! https://jovian.com/alfiya-a/02-insurance-linear-regression


'https://jovian.com/alfiya-a/02-insurance-linear-regression'

## Step 3: Create a Linear Regression Model

Our model itself is a fairly straightforward linear regression (we'll build more complex models in the next assignment). 


In [32]:
input_size = len(input_cols)
output_size = len(output_cols)
print(input_size)
print(output_size)

5
1


**Q9: Complete the class definition below by filling out the constructor (`__init__`), `forward`, `training_step` and `validation_step` methods.**

Hint: Think carefully about picking a good loss fuction (it's not cross entropy). Maybe try 2-3 of them and see which one works best. See https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [33]:
class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(5, 1)                 # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        out = self.linear(xb)                          # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.mse_loss(out, targets)                          # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.mse_loss(out, targets)                       # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

Let us create a model using the `InsuranceModel` class. You may need to come back later and re-run the next cell to reinitialize the model, in case the loss becomes `nan` or `infinity`.

In [34]:
model = InsuranceModel()

Let's check out the weights and biases of the model using `model.parameters`.

In [35]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.3452, -0.2341,  0.2532, -0.1744,  0.0187]], requires_grad=True),
 Parameter containing:
 tensor([-0.2166], requires_grad=True)]

One final commit before we train the model.

In [36]:
jovian.commit(project=project_name, environment=None)

<IPython.core.display.Javascript object>

[jovian] Updating notebook "alfiya-a/02-insurance-linear-regression" on https://jovian.com
[jovian] Committed successfully! https://jovian.com/alfiya-a/02-insurance-linear-regression


'https://jovian.com/alfiya-a/02-insurance-linear-regression'

## Step 4: Train the model to fit the data

To train our model, we'll use the same `fit` function explained in the lecture. That's the benefit of defining a generic training loop - you can use it for any problem.

In [37]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history


**Q10: Use the `evaluate` function to calculate the loss on the validation set before training.**

In [38]:
result = evaluate(model, val_loader) # Use the the evaluate function
print(result)

{'val_loss': 308212768.0}



We are now ready to train the model. You may need to run the training loop many times, for different number of epochs and with different learning rates, to get a good result. Also, if your loss becomes too large (or `nan`), you may have to re-initialize the model by running the cell `model = InsuranceModel()`. Experiment with this for a while, and try to get to as low a loss as possible.

**Q11: Train the model 4-5 times with different learning rates & for different number of epochs.**

Hint: Vary learning rates by orders of 10 (e.g. `1e-2`, `1e-3`, `1e-4`, `1e-5`, `1e-6`) to figure out what works.

In [59]:
model= InsuranceModel()

In [60]:
epochs = 500
lr = 1e-6
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 123563896.0000
Epoch [40], val_loss: 123365600.0000
Epoch [60], val_loss: 123227904.0000
Epoch [80], val_loss: 123305336.0000
Epoch [100], val_loss: 123079936.0000
Epoch [120], val_loss: 122915552.0000
Epoch [140], val_loss: 122711968.0000
Epoch [160], val_loss: 122747400.0000
Epoch [180], val_loss: 122491352.0000
Epoch [200], val_loss: 122949448.0000
Epoch [220], val_loss: 122263040.0000
Epoch [240], val_loss: 121859224.0000
Epoch [260], val_loss: 121782744.0000
Epoch [280], val_loss: 121836696.0000
Epoch [300], val_loss: 121543720.0000
Epoch [320], val_loss: 121336032.0000
Epoch [340], val_loss: 121178936.0000
Epoch [360], val_loss: 121035528.0000
Epoch [380], val_loss: 120811112.0000
Epoch [400], val_loss: 120710872.0000
Epoch [420], val_loss: 120563720.0000
Epoch [440], val_loss: 120550120.0000
Epoch [460], val_loss: 120516632.0000
Epoch [480], val_loss: 120099448.0000
Epoch [500], val_loss: 119774216.0000


In [61]:
epochs = 500
lr = 1e-4
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 116248360.0000
Epoch [40], val_loss: 93700448.0000
Epoch [60], val_loss: 88425272.0000
Epoch [80], val_loss: 89628584.0000
Epoch [100], val_loss: 202328720.0000
Epoch [120], val_loss: 65910588.0000
Epoch [140], val_loss: 96467320.0000
Epoch [160], val_loss: 55369108.0000
Epoch [180], val_loss: 60367628.0000
Epoch [200], val_loss: 48455968.0000
Epoch [220], val_loss: 45851056.0000
Epoch [240], val_loss: 44293212.0000
Epoch [260], val_loss: 42795480.0000
Epoch [280], val_loss: 41395984.0000
Epoch [300], val_loss: 96277184.0000
Epoch [320], val_loss: 43205488.0000
Epoch [340], val_loss: 121937664.0000
Epoch [360], val_loss: 37477652.0000
Epoch [380], val_loss: 36599196.0000
Epoch [400], val_loss: 37532752.0000
Epoch [420], val_loss: 50893120.0000
Epoch [440], val_loss: 72352408.0000
Epoch [460], val_loss: 43513332.0000
Epoch [480], val_loss: 89805504.0000
Epoch [500], val_loss: 34232452.0000


In [62]:
epochs = 500
lr = 1e-4
history3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 35393156.0000
Epoch [40], val_loss: 34809016.0000
Epoch [60], val_loss: 36014080.0000
Epoch [80], val_loss: 34383500.0000
Epoch [100], val_loss: 36857908.0000
Epoch [120], val_loss: 54148980.0000
Epoch [140], val_loss: 33418760.0000
Epoch [160], val_loss: 33111714.0000
Epoch [180], val_loss: 33450240.0000
Epoch [200], val_loss: 34585820.0000
Epoch [220], val_loss: 64530688.0000
Epoch [240], val_loss: 34447704.0000
Epoch [260], val_loss: 39859700.0000
Epoch [280], val_loss: 32873854.0000
Epoch [300], val_loss: 32904816.0000
Epoch [320], val_loss: 33022426.0000
Epoch [340], val_loss: 33155294.0000
Epoch [360], val_loss: 32654992.0000
Epoch [380], val_loss: 32943738.0000
Epoch [400], val_loss: 33745668.0000
Epoch [420], val_loss: 32744080.0000
Epoch [440], val_loss: 37012252.0000
Epoch [460], val_loss: 32357744.0000
Epoch [480], val_loss: 35498792.0000
Epoch [500], val_loss: 32255456.0000


**Q12: What is the final validation loss of your model?**

In [63]:
#model is sensitive to learning rates but gives nan values after trying to increase more than 1e-4, even after re-initialising the model
#loss doesn't seem to decrease further than 32e6
val_loss = evaluate(model, val_loader)
val_loss

{'val_loss': 32255456.0}

Let's log the final validation loss to Jovian and commit the notebook

In [64]:
jovian.log_metrics(val_loss=val_loss)

[jovian] Metrics logged.


In [65]:
jovian.commit(project=project_name, environment=None)

<IPython.core.display.Javascript object>

[jovian] Updating notebook "alfiya-a/02-insurance-linear-regression" on https://jovian.com
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.com/alfiya-a/02-insurance-linear-regression


'https://jovian.com/alfiya-a/02-insurance-linear-regression'

Now scroll back up, re-initialize the model, and try different set of values for batch size, number of epochs, learning rate etc. Commit each experiment and use the "Compare" and "View Diff" options on Jovian to compare the different results.

## Step 5: Make predictions using the trained model

**Q13: Complete the following function definition to make predictions on a single input**

In [67]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)                # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [68]:
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([48.0000,  0.0000, 30.1644,  4.0000,  0.0000])
Target: tensor([11235.4785])
Prediction: tensor([11197.2969])


In [69]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([37.0000,  0.0000, 27.5994,  1.0000,  1.0000])
Target: tensor([20702.8008])
Prediction: tensor([31020.2578])


In [70]:
input, target = val_ds[23]
predict_single(input, target, model)

Input: tensor([34.0000,  0.0000, 40.3218,  2.0000,  0.0000])
Target: tensor([6109.3140])
Prediction: tensor([9030.2559])


In [78]:
input, target = val_ds[67]
predict_single(input, target, model)

Input: tensor([57.0000,  1.0000, 34.0632,  0.0000,  0.0000])
Target: tensor([11580.2920])
Prediction: tensor([11945.3164])


Are you happy with your model's predictions? Try to improve them further.

## (Optional) Step 6: Try another dataset & blog about it

While this last step is optional for the submission of your assignment, we highly recommend that you do it. Try to replicate this notebook for a different linear regression or logistic regression problem. This will help solidify your understanding, and give you a chance to differentiate the generic patterns in machine learning from problem-specific details.You can use one of these starer notebooks (just change the dataset):

- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

Here are some sources to find good datasets:

- https://lionbridge.ai/datasets/10-open-datasets-for-linear-regression/
- https://www.kaggle.com/rtatman/datasets-for-regression-analysis
- https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table
- https://people.sc.fsu.edu/~jburkardt/datasets/regression/regression.html
- https://archive.ics.uci.edu/ml/datasets/wine+quality
- https://pytorch.org/docs/stable/torchvision/datasets.html

We also recommend that you write a blog about your approach to the problem. Here is a suggested structure for your post (feel free to experiment with it):

- Interesting title & subtitle
- Overview of what the blog covers (which dataset, linear regression or logistic regression, intro to PyTorch)
- Downloading & exploring the data
- Preparing the data for training
- Creating a model using PyTorch
- Training the model to fit the data
- Your thoughts on how to experiment with different hyperparmeters to reduce loss
- Making predictions using the model

As with the previous assignment, you can [embed Juptyer notebook cells & outputs from Jovian](https://medium.com/jovianml/share-and-embed-jupyter-notebooks-online-with-jovian-ml-df709a03064e) into your blog. 

Don't forget to share your work on the forum: https://jovian.ai/forum/t/linear-regression-and-logistic-regression-notebooks-and-blog-posts/14039

In [80]:
jovian.commit(project=project_name, environment=None)
jovian.commit(project=project_name, environment=None) # try again, kaggle fails sometimes

<IPython.core.display.Javascript object>

[jovian] Updating notebook "alfiya-a/02-insurance-linear-regression" on https://jovian.com
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.com/alfiya-a/02-insurance-linear-regression


<IPython.core.display.Javascript object>

[jovian] Updating notebook "alfiya-a/02-insurance-linear-regression" on https://jovian.com
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.com/alfiya-a/02-insurance-linear-regression


'https://jovian.com/alfiya-a/02-insurance-linear-regression'